# Processo di Markov di primo ordine {#sec-appendix-markov-first-order}

Il processo di Markov di primo ordine è un concetto fondamentale in vari campi, tra cui l'intelligenza artificiale, la statistica e la teoria delle probabilità. Esso descrive una sequenza di eventi in cui la probabilità di ogni evento dipende solo dallo stato immediatamente precedente, non dall'intera sequenza di eventi passati. Questa caratteristica è nota come "assunzione di Markov".

In un processo di Markov di primo ordine, la probabilità condizionale di un evento è data da:

$$ P(w_n | w_{n-1}, w_{n-2}, \dots, w_1) \approx P(w_n | w_{n-1}). $$

Questa semplificazione riduce significativamente il numero di probabilità da calcolare, rendendo il modello più gestibile.

Per illustrare il concetto, consideriamo un modello di previsione del tempo a Firenze, con tre possibili condizioni: soleggiato, piovoso e nebbioso. Ecco una matrice di transizione che rappresenta le probabilità di passaggio da un tipo di tempo all'altro:

| Tempo Oggi | Soleggiato | Piovoso | Nebbioso |
|------------|------------|---------|----------|
| **Soleggiato** | 0.8        | 0.15    | 0.05     |
| **Piovoso**    | 0.2        | 0.6     | 0.2      |
| **Nebbioso**   | 0.2        | 0.3     | 0.5      |

Calcoliamo alcune probabilità.

- Probabilità di due giorni consecutivi (soleggiato, poi piovoso) dato che oggi è soleggiato:

   $$ P(\text{Soleggiato domani, Piovoso dopodomani} \mid \text{Soleggiato oggi}) = 0.8 \times 0.15 = 0.12 .$$

- Probabilità di pioggia tra due giorni, dato che oggi è nebbioso:

   $$ P(\text{Piovoso dopodomani} \mid \text{Nebbioso oggi}) = 0.5 \times 0.2 + 0.3 \times 0.6 + 0.2 \times 0.15 = 0.31 .$$

Vediamo ora come implementare un processo di Markov di primo ordine in Stan. Iniziamo con il modello meteo discusso sopra.

```stan
data {
  int<lower=1> N;           // Numero di osservazioni
  int<lower=1> K;           // Numero di stati possibili (3 per Soleggiato, Piovoso, Nebbioso)
  int<lower=1, upper=K> y[N]; // Sequenza di osservazioni (stati)
}

parameters {
  simplex[K] P[K]; // Matrice di transizione delle probabilità
}

model {
  for (n in 2:N) {
    y[n] ~ categorical(P[y[n-1]]);
  }
}
```

In questo modello, usiamo la distribuzione `categorical()` per modellare le transizioni tra i tre stati meteorologici possibili.

Consideriamo ora una situazione ancora più semplice, come quella discussa nel @sec-post-error. In questo secondo caso, abbiamo una sequenza di prove con solo due stati: prova prova corretta (0) e prova sbagliata (1). Il nostro interesse è la prestazione post-errore. Questo scenario si presta naturalmente all'uso della distribuzione `bernoulli()`.

```stan
data {
  int<lower=1> N;           // Numero di prove
  int<lower=0, upper=1> y[N]; // Sequenza di risultati (0 = corretto, 1 = errore)
}

parameters {
  real<lower=0, upper=1> p_error_given_correct;
  real<lower=0, upper=1> p_error_given_error;
}

model {
  for (n in 2:N) {
    if (y[n - 1] == 1) {
      y[n] ~ bernoulli(p_error_given_error);
    } else {
      y[n] ~ bernoulli(p_error_given_correct);
    }
  }
}
```

In questo modello:

- `p_error_given_correct` rappresenta la probabilità di errore dopo una prova corretta.
- `p_error_given_error` rappresenta la probabilità di errore dopo un errore precedente.

Mentre nel modello Meteo avevamo tre stati (Soleggiato, Piovoso, Nebbioso), nel caso del modello Post-Errore abbiamo solo due stati (Corretto, Errore). Ciò porta ad una differenza tra le funzioni di verosimiglianza che vengono utilizzate:

- Modello Meteo: `categorical()`;
- Modello Post-Errore: `bernoulli()`.

Di conseguenza, la parametrizzazione dei modelli cambia:

   - Modello Meteo: Matrice di transizione completa (`simplex[K] P[K]`);
   - Modello Post-Errore: Due probabilità di transizione (`p_error_given_correct`, `p_error_given_error`)

In conclusione, il processo di Markov di primo ordine è uno strumento potente per modellare sequenze di eventi con memoria limitata. L'implementazione in Stan permette di effettuare inferenze su questi modelli, sia in contesti con molteplici stati (come le previsioni meteo) sia in scenari binari (come l'analisi della prestazione post-errore). 